In [1]:
from ultralytics import YOLO
import pyrealsense2 as rs
from PIL import Image
import numpy as np
import ultralytics
import math
import cv2
import vision_functions

In [2]:
model = YOLO("YOLOv8sNO.pt")

# Webcam

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
while cap.isOpened():
    success, frame = cap.read()
    assert success

    res = model.track(frame, persist=True)
    annot_frame = res[0].plot()

    cv2.imshow("Tracking", annot_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 299.6ms
Speed: 4.7ms preprocess, 299.6ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""



0: 480x640 (no detections), 332.2ms
Speed: 14.6ms preprocess, 332.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 208.5ms
Speed: 2.3ms preprocess, 208.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 189.4ms
Speed: 1.7ms preprocess, 189.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 230.0ms
Speed: 1.5ms preprocess, 230.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 242.3ms
Speed: 3.0ms preprocess, 242.3ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 223.6ms
Speed: 5.2ms preprocess, 223.6ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 188.8ms
Speed: 3.0ms preprocess, 188.8ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 200.5ms
Speed: 3.0ms prep

In [ ]:
cap.release()
cv2.destroyAllWindows()

# Realsense

In [104]:
pipe = rs.pipeline()
aligner = rs.align(rs.stream.color)
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

profile = pipe.start(config)

In [105]:
while True:
    frames = pipe.wait_for_frames()
    #frames = aligner.process(frames)
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()
    
    color_im = np.asanyarray(color_frame.get_data())
    processing_im = color_im.copy()
    #cv2.cvtColor(color_im, cv2.COLOR_RGB2BGR, color_im)
    depth_im = np.asanyarray(depth_frame.get_data())

    result = model.track(processing_im, persist=True)
    annot_frame = result[0].plot()

    if (result.__len__() >= 1):
        #x1, y1, x2, y2 = (int, int, int, int)
        x1, y1, x2, y2 = result[0].boxes.xyxy.numpy()
        
        bounding_box = color_im[y1:y2, x1:x2]
        cv2.imshow("Target",bounding_box)

    cv2.imshow("Tracking",annot_frame[...,[2,1,0]])
    cv2.imshow("Depth", depth_im)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()


0: 480x640 1 note, 168.4ms
Speed: 4.2ms preprocess, 168.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)


IndexError: index 1 is out of bounds for dimension 0 with size 1